<b>Install required dependencies and libraries</b>

In [1]:
! pip install lxml html5lib beautifulsoup4

In [2]:
import pandas as pd
import numpy as np

<b>Extract Data Table from URL</b>

In [3]:
#Webpage URL
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Extract Tables
dfs = pd.read_html(url)

#Get the first table
df=dfs[0]

#Extract Columns
dfYTO = df[['Postal Code','Borough', 'Neighbourhood']]
print(dfYTO)        

    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                         Neighbourhood  
0                                         Not assigned  
1                                         Not assigned  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
..                                                 ...  
175                                       Not assigned  
176                                       Not assigned  
177                                       

In [4]:
#Check initial entries:
dfYTO.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
#Drop the Borough Rows showing 'Not Assigned'  and save as a new data frame
dfYTO1 = dfYTO[dfYTO.Borough != 'Not assigned']

In [10]:
dfYTO1.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<b>I have used the duplicated function to find out if there was any duplicated Postal Code in the dataframe:</b>

In [11]:
#Command to generate a Dupe Dataframe only for the column Postal Code
PCdupe=dfYTO1['Postal Code'].duplicated()

In [12]:
#Check the PCdupe dataframe: 
PCdupe.head()

2    False
3    False
4    False
5    False
6    False
Name: Postal Code, dtype: bool

In [13]:
#I will also do a value count, only values showing as True would be duplicated: 
PCdupe.value_counts()

False    103
Name: Postal Code, dtype: int64

My finding after the counting and the duplicated function, indicates that there are not duplicated postal codes on the main dataframe dfYTO1. 

<b>Now, I will look for any <i>Not assigned</i> variables in the Neighbourhood column.</b>

In [14]:
#Use .loc function look up for any not assigned variables: 
dfnhood=dfYTO1.loc[dfYTO1['Neighbourhood']=='Not assigned']

In [15]:
dfnhood

,Postal Code,Borough,Neighbourhood


As per the above there are no cells showing as Not Assigned in the current dfYTO1 dataframe.

In [17]:
#shows the shape of my dataframe as it stands now: 
dfYTO1.shape

(103, 3)

After the initial data processing. I have 103 rows and 3 columns.

<b> Now we are going to find the latitude and longitude of the locations to work with Four Square:</b>

When trying to load Geocoder I had some compatibility issues, so my choice was to use the provided file and combine the databases. 

In [18]:
Geo = pd.read_csv("https://cocl.us/Geospatial_data")
print(Geo)

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
..          ...        ...        ...
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054

[103 rows x 3 columns]


In order to make sure the information is added to the correct postal code, I will use an inner join to combine both databases into a new one. My joining variable being the Postal Code.

In [20]:
YTOGeo = pd.merge(left=dfYTO1, right=Geo, left_on='Postal Code', right_on='Postal Code')

In [21]:
YTOGeo.shape

(103, 5)

In [22]:
YTOGeo.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


As per the above, after merging dfYTO and the Geo dataframes, into a new dataframe called YTOGeo, I still landed with 103 rows and 5 columns and when verifying the new dataframe YTOGeo, we have now the Latitude and Longitude for all locations. 